# BioInformatcis Project 5

## Import python packages

In [1]:
# %matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import scipy.sparse as sp

from utils import *
from model import *

## Preprocess data

In [2]:
adj = load_data()
num_nodes = adj.shape[0]
num_edges = adj.sum()

# Featureless
features = sparse2tensor(sp.identity(num_nodes))
num_features = num_nodes
features_nonzero = num_nodes

print("Data load Success!!")

adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false, orig_matrix = split_data(adj)
adj_train_norm = normalize_graph(adj_train)

Data load Success!!
Data split Success!!


## Create & Fit model

In [3]:
# Create model
model = GCN(num_features, features_nonzero, num_h1=32, num_h2=16, dropout=0.1)

In [4]:
# Fit model
adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse2tensor(adj_label)

In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
pos_weight = (num_nodes**2 - num_edges) / num_edges
norm = num_nodes ** 2 / ((num_nodes**2 - num_edges) * 2)    
    
epochs=100
for epoch in range(epochs):
    #print('Start of epoch %d' % (epoch,))
    inputs = [features, adj_train_norm]
    
    with tf.GradientTape() as tape:
        recon = model(inputs, training=True)        
        loss =  norm * tf.reduce_mean(
                        tf.nn.weighted_cross_entropy_with_logits(labels=tf.sparse.to_dense(adj_label),
                                                                logits=recon,
                                                                pos_weight=pos_weight))
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    
    val_roc, val_ap = get_roc_score(recon, real_edges=val_edges, fake_edges=val_edges_false)
    
    print("Epoch:", '%04d' % (epoch + 1), 
          "train_loss=", "{:.5f}".format(loss),
          "val_roc=", "{:.5f}".format(val_roc),
          "val_ap=", "{:.5f}".format(val_ap))

print("Training is Finished")
    
test_roc, test_ap = get_roc_score(recon, real_edges=test_edges, fake_edges=test_edges_false)  
print("Epoch:", 'finished', 
      "test_roc=", "{:.5f}".format(val_roc),
      "test_ap=", "{:.5f}".format(val_ap),)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 0001 train_loss= 0.68107 val_roc= 0.76183 val_ap= 0.68914
Epoch: 0002 train_loss= 0.66503 val_roc= 0.86738 val_ap= 0.85808
Epoch: 0003 train_loss= 0.61654 val_roc= 0.87221 val_ap= 0.86111
Epoch: 0004 train_loss= 0.70334 val_roc= 0.87312 val_ap= 0.86290
Epoch: 0005 train_loss= 0.62391 val_roc= 0.87184 val_ap= 0.86088
Epoch: 0006 train_loss= 0.61635 val_roc= 0.87112 val_ap= 0.85969
Epoch: 0007 train_loss= 0.63869 val_roc= 0.87138 val_ap= 0.86153
Epoch: 0008 train_loss= 0.64838 val_roc= 0.87213 val_